In [1]:
!pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 50.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 1.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 87.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 82.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 552.0/552.0 kB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 22.3 MB/s eta 0:00:00
   

In [3]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)


Found the config file in: /config.json


In [15]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import command

job_inputs = {'train_data': Input(type=AssetTypes.URI_FILE, path="azureml:training-data:1"), 
'test_data': Input(type=AssetTypes.URI_FILE, path="azureml:test-data:1"), 
'data_description': Input(type=AssetTypes.URI_FILE, path="azureml:data-desc:1")}

# configure job
job = command(
    code="./src",
    command="python train_model.py --train_data ${{inputs.train_data}} --test_data ${{inputs.test_data}} --data_description ${{inputs.data_description}}",
    inputs = job_inputs,
    environment="AzureML-sklearn-1.5@latest",
    compute="aml-cluster",
    display_name="house-prices-e2e-v2",
    experiment_name="house-prices-e2e-v2-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Uploading src (0.02 MBs): 100%|██████████| 23579/23579 [00:00<00:00, 151371.94it/s]




Monitor your job at https://ml.azure.com/runs/plum_receipt_clkr0t0lq6?wsid=/subscriptions/ad5c3a0e-2e80-43f8-a0e0-2cb6463d9e1e/resourcegroups/rg-house-prices-e2e/workspaces/mlw-house-prices-e2e-d18c90063d&tid=6f2cb654-d755-4b1d-b2c4-b18e5c3e2442


In [11]:
import mlflow

In [12]:
experiment_name = "mlflow-experiment-diabetes"
mlflow.set_experiment(experiment_name)

2024/09/04 21:24:24 INFO mlflow.tracking.fluent: Experiment with name 'mlflow-experiment-diabetes' does not exist. Creating a new experiment.


<Experiment: artifact_location='', creation_time=1725485064736, experiment_id='8adbee66-01fc-4902-84fc-6d4d27a8a2ed', last_update_time=None, lifecycle_stage='active', name='mlflow-experiment-diabetes', tags={}>

In [13]:
with mlflow.start_run():
    mlflow.log_metric("Accuracy", 0.3)

2024/09/04 21:24:47 INFO mlflow.tracking._tracking_service.client: 🏃 View run shy_evening_tbtngfx6 at: https://westeurope.api.azureml.ms/mlflow/v2.0/subscriptions/ad5c3a0e-2e80-43f8-a0e0-2cb6463d9e1e/resourceGroups/rg-house-prices-e2e/providers/Microsoft.MachineLearningServices/workspaces/mlw-house-prices-e2e-d18c90063d/#/experiments/8adbee66-01fc-4902-84fc-6d4d27a8a2ed/runs/402cb119-ab02-46ef-8480-0d729d3da50e.
2024/09/04 21:24:47 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://westeurope.api.azureml.ms/mlflow/v2.0/subscriptions/ad5c3a0e-2e80-43f8-a0e0-2cb6463d9e1e/resourceGroups/rg-house-prices-e2e/providers/Microsoft.MachineLearningServices/workspaces/mlw-house-prices-e2e-d18c90063d/#/experiments/8adbee66-01fc-4902-84fc-6d4d27a8a2ed.
